In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv(r'E:\ABD-work\ScopusID_part5.csv' , low_memory=False , nrows = 100) # Give the path where your excel is.
df.drop('Unnamed: 0' , axis = 1 , inplace=True)
df.head()

,Enumeration_date,NPI,Sole_proprietor,Taxonomies_code,Taxonomies_desc,First_name,Last_name,Mid_name,Location_country_code,Location_address1,...,Mailing_address2,Mailing_city,Mailing_state,Mailing_postal_code,Mailing_telephone_number,Mailing_fax_number,OrcidID_Exact,OrcidID_Fuzzy,Orcid_MatchType,Orcid_NumOfResults
0,2021-05-15,1942874896,NO,390200000X,Student in an Organized Health Care Education/...,GUILLERMO,RODRIGUEZ,NaN,US,3990 JOHN R STREET,...,ORTHO ADMIN 7 BRUSH,TAMARAC,FL,333218803.0,954-288-6058,NaN,NaN,NaN,Too_Fuzzy,87.0
1,2021-05-16,1063086932,NO,390200000X,Student in an Organized Health Care Education/...,EMILIA,MIRONOVICI,NaN,US,707 E MAIN ST,...,NaN,MIDDLETOWN,NY,109402650.0,845-333-7373,845-333-7342,NaN,NaN,No Match,0.0
2,2021-05-16,1083288948,NaN,343900000X,Non-emergency Medical Transport (VAN),Jean,Noel,M,US,816 W LIBERTY ST,...,NaN,ALLENTOWN,PA,181025524.0,484-773-3029,NaN,NaN,NaN,Too_Fuzzy,8.0
3,2021-05-16,1104490077,YES,106E00000X,Assistant Behavior Analyst,SHANDY,SKAGGS,LATOYA,US,1529 SAM RITTENBERG BLVD STE 1B,...,NaN,CHARLESTON,SC,294074125.0,888-547-2250,NaN,NaN,NaN,No Match,0.0
4,2021-05-16,1154995017,NaN,207RR0500X,"Internal Medicine, Rheumatology",Diana,Girnita,M,US,877 W FREMONT AVE STE N1,...,NaN,PALO ALTO,CA,943064548.0,513-917-0908,650-263-7265,0000-0001-8976-176X,NaN,Exact,1.0


In [3]:
df['OrcidID_Exact'].value_counts()

0000-0001-8976-176X    1
0000-0002-5812-2487    1
0000-0001-5150-782X    1
0000-0002-7161-8970    1
0000-0001-8955-850X    1
0000-0002-5230-2065    1
0000-0003-2802-0823    1
0000-0001-7642-0085    1
0000-0002-0372-2282    1
Name: OrcidID_Exact, dtype: int64

In [4]:
df.shape

(100, 28)

In [5]:
df['ScopusID_Exact'] = np.nan
df['ScopusID_Fuzzy'] = np.nan
df['Scopus_MatchType'] = np.nan
df['Scopus_NumOfResults'] = np.nan

In [6]:
# df['ScopusID'] = df['ScopusID'].astype('object')

In [7]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
} 

In [8]:
def get_scopus_id_wOrcid(orcid):
    try : 
        orcid = orcid
        r = requests.get(f"https://www.scopus.com/results/authorNamesList.uri?sort=count-f&src=al&orcidId={orcid}&sid=b7605a2c701a14c0d136648699739b87&sot=al&sdt=al&sl=36&s=AUTH--ORCID--ID%28{orcid}%29&selectionPageSearch=anl&reselectAuthor=false&activeFlag=true&showDocument=false&resultsPerPage=20&offset=1&jtp=false&currentPage=1&previousSelectionCount=0&tooManySelections=false&previousResultCount=0&authSubject=LFSC&authSubject=HLSC&authSubject=PHSC&authSubject=SOSC&exactAuthorSearch=false&showFullList=false&authorPreferredName=&origin=searchauthorfreelookup&affiliationId=&txGid=e4f338e3dab473a40424b29187e91bb0" , 
                 headers=headers)
        # exactAuthorSearch=false
        html = r.text
        soup = BeautifulSoup(html , 'html.parser')
        scopus_id = soup.find_all('input' , id = 'pageItems' , value = lambda x : x and x.isnumeric())[0]['value']
        return scopus_id
    except : 
        return []

In [9]:
def get_scopus_id_wName(name, last_name):
    try:
        # r = requests.get(f"https://www.scopus.com/results/authorNamesList.uri?sort=count-f&src=al&affilName={affilication}*&sid=28a196b5714eefc7b7b3e5ccdd92684e&sot=al&sdt=al&sl=55&s=AUTHLASTNAME%28{last_name}%29+AND+AUTHFIRST%28{name}%29+AND+AFFIL%28{affilication}*%29&st1={last_name}&st2=David&orcidId=&selectionPageSearch=anl&reselectAuthor=false&activeFlag=true&showDocument=false&resultsPerPage=20&offset=1&jtp=false&currentPage=1&previousSelectionCount=0&tooManySelections=false&previousResultCount=0&authSubject=LFSC&authSubject=HLSC&authSubject=PHSC&authSubject=SOSC&exactAuthorSearch=false&showFullList=false&authorPreferredName=&origin=searchauthorfreelookup&affiliationId=&txGid=8ffbb8e5e5ac9e721b4f02e2f80f1f02", 
        #                  headers=headers)
        r = requests.get(f"https://www.scopus.com/results/authorNamesList.uri?sort=count-f&src=al&sid=9cd1473165b827e1d1638ea8f17dc68e&sot=al&sdt=al&sl=39&s=AUTHLASTNAME%28{last_name}%29+AND+AUTHFIRST%28{name}%29&st1={last_name}&st2={name}&orcidId=&selectionPageSearch=anl&reselectAuthor=false&activeFlag=true&showDocument=false&resultsPerPage=20&offset=1&jtp=false&currentPage=1&previousSelectionCount=0&tooManySelections=false&previousResultCount=0&authSubject=LFSC&authSubject=HLSC&authSubject=PHSC&authSubject=SOSC&exactAuthorSearch=true&showFullList=false&authorPreferredName=&origin=searchauthorfreelookup&affiliationId=&txGid=07e19e45cb76e734106475e73b349492", 
                         headers=headers) # For only name and last_name
        html = r.text
        soup = BeautifulSoup(html, 'html.parser')
        scopus_ids = soup.find_all('input', value=lambda x: x and x.isnumeric() and (len(x) == 11 or len(x) == 10))
        if len(scopus_ids) == 1:
            return scopus_ids[0]['value']
        elif len(scopus_ids) > 1:
            ids = [id['value'] for id in scopus_ids]
            return list(set(ids))
        else :
            return []
    except:
        return []

In [10]:
orcid_count = 0
name_0_count = 0
name_1_count = 0
name_many_count = 0
name_many_many_count = 0
else1_count = 0
else2_count = 0

In [11]:
df['Orcid_MatchType'].value_counts(dropna=False)

No Match     66
Too_Fuzzy    13
Fuzzy        12
Exact         9
Name: Orcid_MatchType, dtype: int64

In [12]:
df.head()

,Enumeration_date,NPI,Sole_proprietor,Taxonomies_code,Taxonomies_desc,First_name,Last_name,Mid_name,Location_country_code,Location_address1,...,Mailing_telephone_number,Mailing_fax_number,OrcidID_Exact,OrcidID_Fuzzy,Orcid_MatchType,Orcid_NumOfResults,ScopusID_Exact,ScopusID_Fuzzy,Scopus_MatchType,Scopus_NumOfResults
0,2021-05-15,1942874896,NO,390200000X,Student in an Organized Health Care Education/...,GUILLERMO,RODRIGUEZ,NaN,US,3990 JOHN R STREET,...,954-288-6058,NaN,NaN,NaN,Too_Fuzzy,87.0,NaN,NaN,NaN,NaN
1,2021-05-16,1063086932,NO,390200000X,Student in an Organized Health Care Education/...,EMILIA,MIRONOVICI,NaN,US,707 E MAIN ST,...,845-333-7373,845-333-7342,NaN,NaN,No Match,0.0,NaN,NaN,NaN,NaN
2,2021-05-16,1083288948,NaN,343900000X,Non-emergency Medical Transport (VAN),Jean,Noel,M,US,816 W LIBERTY ST,...,484-773-3029,NaN,NaN,NaN,Too_Fuzzy,8.0,NaN,NaN,NaN,NaN
3,2021-05-16,1104490077,YES,106E00000X,Assistant Behavior Analyst,SHANDY,SKAGGS,LATOYA,US,1529 SAM RITTENBERG BLVD STE 1B,...,888-547-2250,NaN,NaN,NaN,No Match,0.0,NaN,NaN,NaN,NaN
4,2021-05-16,1154995017,NaN,207RR0500X,"Internal Medicine, Rheumatology",Diana,Girnita,M,US,877 W FREMONT AVE STE N1,...,513-917-0908,650-263-7265,0000-0001-8976-176X,NaN,Exact,1.0,NaN,NaN,NaN,NaN


In [13]:
for index, row in df.iterrows():
    if row['First_name'].isna():
        continue
    else :
        name = row['First_name']
        last_name = row['Last_name']
        
        if row['Orcid_MatchType'] == 'Exact' : 
            orcid = row['OrcidID_Exact']
            for i in range(2):
                scopus_id = get_scopus_id_wOrcid(orcid)
                if len(scopus_id) == 1 : 
                    df.at[index, 'ScopusID'] = scopus_id
                    orcid_count += 1
                    break
                else :
                    else1_count += 1
                    
                scopus_id = get_scopus_id_wName(name,last_name)

                if len(scopus_id) == 0 :
                    df.at[index, 'Scopus_MatchType'] = 'No Match'
                    df.at[index, 'Scopus_NumOfResults'] = 0
                    name_0_count += 1
                    break
                elif len(scopus_id) == 1 :
                    df.at[index, 'ScopusID_Exact'] = scopus_id[0]
                    df.at[index, 'Scopus_MatchType'] = 'Exact'
                    df.at[index, 'Scopus_NumOfResults'] = 1
                    name_1_count += 1
                    break
                elif 1 < len(scopus_id) <= 5 :
                    df.at[index, 'ScopusID_Fuzzy'] = scopus_id
                    df.at[index, 'Scopus_MatchType'] = 'Fuzzy'
                    df.at[index, 'Scopus_NumOfResults'] = len(scopus_id)
                    name_many_count += 1
                    break
                elif 5 < len(scopus_id) :
                    df.at[index, 'Scopus_MatchType'] = 'Too_Fuzzy'
                    df.at[index, 'Scopus_NumOfResults'] = len(scopus_id)
                    name_many_many_count += 1
                    break
                else : 
                    else2_count += 1
                
        else : 
            for i in range(2):
                scopus_id = get_scopus_id_wName(name,last_name)
                if len(scopus_id) == 0 :
                    df.at[index, 'Scopus_MatchType'] = 'No Match'
                    df.at[index, 'Scopus_NumOfResults'] = 0
                    name_0_count += 1
                    break
                elif len(scopus_id) == 1 :
                    df.at[index, 'ScopusID_Exact'] = scopus_id[0]
                    df.at[index, 'Scopus_MatchType'] = 'Exact'
                    df.at[index, 'Scopus_NumOfResults'] = 1
                    name_1_count += 1
                    break
                elif 1 < len(scopus_id) <= 5 :
                    df.at[index, 'ScopusID_Fuzzy'] = scopus_id
                    df.at[index, 'Scopus_MatchType'] = 'Fuzzy'
                    df.at[index, 'Scopus_NumOfResults'] = len(scopus_id)
                    name_many_count += 1
                    break
                elif 5 < len(scopus_id) :
                    df.at[index, 'Scopus_MatchType'] = 'Too_Fuzzy'
                    df.at[index, 'Scopus_NumOfResults'] = len(scopus_id)
                    name_many_many_count += 1
                    break

In [ ]:
df['Scopus_MatchType'].value_counts()

In [ ]:
df.to_excel('ScopusID_Results_part1.xlsx' , sep = '^')